In [1]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import pandas as pd

from TinyTNAS import TinyTNAS
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from ModelBank import *


from utils.data_preparation import *
from utils.preprocessing import *
from utils.features import *

2025-03-31 11:40:41.177437: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /tmp/.mount_cursorLlI9g5/usr/lib:
2025-03-31 11:40:41.177456: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
path = '/home/bmis/Documents/AI-Workspace/sEMGClassification/AdaptiveModel/data/6_Flex_BMIS/flex_bmis/mat_data'
#path ='/home/bmis/Documents/AI-Workspace/sEMGClassification/AdaptiveModel/data/2_BMIS/mat_data'
subject_number = 2
session_number = 1
record_time = 5

train_repetition = [1, 2, 3, 4, 5, 6, 7, 8, 9]
test_repetition = [3, 5]

gesture = [1, 2, 3, 4, 5, 6, 7]
selected_gesture = [1, 2, 3, 4, 5, 6, 7]
num_gesture = len(gesture)

no_channel = 8
low_cut = 10.0
high_cut = 120.0
notch_freq = 60.0
overlap = 60
window_time = 200
order = 5
fs = 250

record_time = 5

start_subject=2
end_subject=4 
session_number=1
activate_session=True

batch_size = 32

In [3]:
num_class = 7
lossf = 0
constraints_specs= {"ram"   : 1024*20,    # 'ram' in bytes
                    "flash" : 1024*64,    # 'flash' in bytes
                    "macc"  : 60*1000 }


emg_prep = EMGDataPreparation(base_path=path, fs=fs, rec_time=record_time)
subject_path, train_gesture, test_gesture = emg_prep.get_per_subject_file(subject_number=subject_number, num_gesture=num_gesture, activate_session=True, session=session_number, train_repetition=train_repetition, test_repetition=test_repetition)

train_data, _ = emg_prep.load_data_per_subject(subject_path, selected_gesture=selected_gesture, train_gesture=train_gesture, test_gesture=test_gesture)
train_data, train_labels = emg_prep.get_data_labels(train_data)

train_data = train_data/10000
train_data = np.abs(train_data)

preprocess = EMGPreprocessing(fs=fs, notch_freq=notch_freq, low_cut=low_cut, high_cut=high_cut, order=order)
train_data = preprocess.remove_mains(train_data)
# test_data = preprocess.remove_mains(test_data)

train_data = preprocess.bandpass_filter(train_data)
#test_data = preprocess.bandpass_filter(test_data)

# train_data = preprocess.highpass_filter(train_data)
# test_data = preprocess.highpass_filter(test_data)

window_train_data, window_train_labels = emg_prep.window_with_overlap(train_data, train_labels, window_time=window_time, overlap=overlap, no_channel=no_channel)

window_train_data, window_train_labels, window_test_data, window_test_labels = data_split(window_train_data, window_train_labels, train_ratio=75)

window_train_data, window_train_labels, window_val_data, window_val_labels = data_split(window_train_data, window_train_labels, train_ratio=70)
# window_test_data, window_test_labels = emg_prep.window_with_overlap(test_data, test_labels, window_time=window_time, overlap=overlap, no_channel=no_channel)


window_train_data, window_train_labels = shuffle_data(window_train_data, window_train_labels)
window_val_data, window_val_labels = shuffle_data(window_val_data, window_val_labels)
window_test_data, window_test_labels = shuffle_data(window_test_data, window_test_labels)

window_train_data = raw_emg_data(window_train_data)
window_val_data = raw_emg_data(window_val_data)
window_test_data = raw_emg_data(window_test_data)

window_train_data.shape, window_train_labels.shape, window_test_data.shape, window_test_labels.shape, window_val_data.shape, window_val_labels.shape

((1983, 8, 50, 1), (1983,), (945, 8, 50, 1), (945,), (851, 8, 50, 1), (851,))

In [4]:
X = window_train_data
Y = window_train_labels

X_val = window_val_data
Y_val = window_val_labels

batch_size=32
input_shape = (X.shape[1:])
learning_rate = 0.0001

train_ds = (X,Y)
val_ds = (X_val,Y_val)


X_test = window_test_data
y_test = window_test_labels

In [5]:
input_size = window_train_data.shape[1:]
input_size

(8, 50, 1)

In [6]:
def emg_net():
    
    inputs = tf.keras.Input(shape=input_size)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu", input_shape=input_size)(inputs)
    x = tf.keras.layers.Conv2D(32, 3, activation="relu")(x)
    x = tf.keras.layers.MaxPool2D((2,2))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(151, activation='relu')(x)
    outputs = tf.keras.layers.Dense(7, activation='softmax')(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name="BMIS-EMG-NET")
    
    return model

In [7]:
model = emg_net()
model.summary()

Model: "BMIS-EMG-NET"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 50, 1)]        0         
                                                                 
 conv2d (Conv2D)             (None, 6, 48, 32)         320       
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 46, 32)         9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 2, 23, 32)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 2, 23, 32)         0         
                                                                 
 flatten (Flatten)           (None, 1472)              0         
                                                      

2025-03-31 11:40:42.857358: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-03-31 11:40:42.868096: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /tmp/.mount_cursorLlI9g5/usr/lib:
2025-03-31 11:40:42.868320: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /tmp/.mount_cursorLlI9g5/usr/lib:
2025-03-31 11:40:42.868508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or 

In [8]:
lr = 0.001
opt = tf.keras.optimizers.Adam(learning_rate=lr, epsilon=1e-07, clipnorm=1)
ls = 'sparse_categorical_crossentropy'
mtr = 'accuracy'

In [9]:
window_train_data.max(), window_train_data.min()

(0.5426898972571771, -0.5316906097043492)

In [10]:
model = emg_net()
model.compile(optimizer=opt, loss=ls, metrics=mtr)
early_stop = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=5)

In [11]:
history = model.fit(window_train_data, window_train_labels, validation_data=(window_val_data, window_val_labels), callbacks=[early_stop],
                    batch_size=32, epochs=30, verbose=1)

Epoch 1/30
62/62 [==============================] - 1s 12ms/step - loss: 1.4746 - accuracy: 0.2769 - val_loss: 5.7094 - val_accuracy: 0.0000e+00
Epoch 2/30
62/62 [==============================] - 1s 9ms/step - loss: 1.3808 - accuracy: 0.2733 - val_loss: 6.1136 - val_accuracy: 0.0000e+00
Epoch 3/30
62/62 [==============================] - 1s 9ms/step - loss: 1.3282 - accuracy: 0.3439 - val_loss: 5.5564 - val_accuracy: 0.0000e+00
Epoch 4/30
62/62 [==============================] - 1s 13ms/step - loss: 1.1457 - accuracy: 0.4846 - val_loss: 6.5114 - val_accuracy: 0.0000e+00
Epoch 5/30
62/62 [==============================] - 1s 12ms/step - loss: 0.9427 - accuracy: 0.6122 - val_loss: 7.2959 - val_accuracy: 0.1751
Epoch 6/30
62/62 [==============================] - 1s 9ms/step - loss: 0.7734 - accuracy: 0.6884 - val_loss: 7.5087 - val_accuracy: 0.1868
Epoch 7/30
62/62 [==============================] - 1s 9ms/step - loss: 0.6579 - accuracy: 0.7504 - val_loss: 7.2126 - val_accuracy: 0.1739
E

In [12]:
loss, accuracy = model.evaluate(window_test_data, window_test_labels)
print(f'The accuracy of the model is {accuracy*100}:.4f')

30/30 [==============================] - 0s 2ms/step - loss: 13.5787 - accuracy: 0.0000e+00
The accuracy of the model is 0.0:.4f
